In [1]:
# coding: UTF-8
import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import os
import gc
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import datetime
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

In [2]:
# 读取数据
train_sales = pd.read_csv('train_sales_data.csv')
train_search = pd.read_csv('train_search_data.csv')
train_user = pd.read_csv('train_user_reply_data.csv')
evaluation_public = pd.read_csv('evaluation_public.csv')
submit_example = pd.read_csv('submit_example.csv')

In [3]:
# 将sales和evaluation_public进行自然连接
data = pd.concat([train_sales, evaluation_public], ignore_index=True, sort=True)
# 将data和train_search在province、adcode、model、regYear、regMonth上进行左连接
data = data.merge(train_search, 'left', on=[
                  'province', 'adcode', 'model', 'regYear', 'regMonth'])
# 将data和train_user在model、regYear、regMonth上进行左连接
data = data.merge(train_user, 'left', on=['model', 'regYear', 'regMonth'])

In [4]:
# 添加一个新列label，其值与salesVolume一样
data['label'] = data['salesVolume']

In [5]:
data.describe()

,adcode,forecastVolum,id,regMonth,regYear,salesVolume,popularity,carCommentVolum,newsReplyVolum,label
count,36960.000000,0.0,5280.000000,36960.000000,36960.000000,31680.000000,31680.000000,31680.000000,31680.000000,31680.000000
mean,347727.272727,NaN,2677.166667,5.928571,2016.714286,608.709122,2311.965941,250.479167,1989.809722,608.709122
std,136281.363156,NaN,1549.503211,3.514595,0.699864,779.076734,2980.547086,318.648963,2324.517694,779.076734
min,110000.000000,NaN,1.000000,1.000000,2016.000000,2.000000,25.000000,0.000000,0.000000,2.000000
25%,230000.000000,NaN,1342.750000,3.000000,2016.000000,182.000000,700.000000,54.000000,335.000000,182.000000
50%,355000.000000,NaN,2684.500000,5.500000,2017.000000,363.000000,1445.500000,149.500000,1432.500000,363.000000
75%,440000.000000,NaN,4026.250000,9.000000,2017.000000,737.000000,2816.000000,307.000000,2712.000000,737.000000
max,610000.000000,NaN,5368.000000,12.000000,2018.000000,15317.000000,116300.000000,2834.000000,20770.000000,15317.000000


In [6]:
data['is'] = data['regMonth'] == 2
data['is'].to_csv('is_spring_featival',index=False)

In [7]:
# 将缺失值填充为均值
for colname in ['adcode','forecastVolum','regMonth','regYear',
'salesVolume','popularity','carCommentVolum','newsReplyVolum','label']:
    data[colname] = data[colname].fillna(data[colname].mean())
data.head()

,adcode,bodyType,forecastVolum,id,model,province,regMonth,regYear,salesVolume,popularity,carCommentVolum,newsReplyVolum,label,is
0,310000,SUV,NaN,NaN,3c974920a76ac9c1,上海,1,2016,292.0,1479.0,11.0,106.0,292.0,False
1,530000,SUV,NaN,NaN,3c974920a76ac9c1,云南,1,2016,466.0,1594.0,11.0,106.0,466.0,False
2,150000,SUV,NaN,NaN,3c974920a76ac9c1,内蒙古,1,2016,257.0,1479.0,11.0,106.0,257.0,False
3,110000,SUV,NaN,NaN,3c974920a76ac9c1,北京,1,2016,408.0,2370.0,11.0,106.0,408.0,False
4,510000,SUV,NaN,NaN,3c974920a76ac9c1,四川,1,2016,610.0,3562.0,11.0,106.0,610.0,False


In [8]:
# 首先去除train_sales中model字段的重复项，然后将model作为bodyType的下标
# 最后将data的bodyType设置为其model字段在map中映射的bodyType的值
data['bodyType'] = data['model'].map(
    train_sales.drop_duplicates('model').set_index('model')['bodyType'])
data.head()

,adcode,bodyType,forecastVolum,id,model,province,regMonth,regYear,salesVolume,popularity,carCommentVolum,newsReplyVolum,label,is
0,310000,SUV,NaN,NaN,3c974920a76ac9c1,上海,1,2016,292.0,1479.0,11.0,106.0,292.0,False
1,530000,SUV,NaN,NaN,3c974920a76ac9c1,云南,1,2016,466.0,1594.0,11.0,106.0,466.0,False
2,150000,SUV,NaN,NaN,3c974920a76ac9c1,内蒙古,1,2016,257.0,1479.0,11.0,106.0,257.0,False
3,110000,SUV,NaN,NaN,3c974920a76ac9c1,北京,1,2016,408.0,2370.0,11.0,106.0,408.0,False
4,510000,SUV,NaN,NaN,3c974920a76ac9c1,四川,1,2016,610.0,3562.0,11.0,106.0,610.0,False


In [9]:
for i in ['bodyType', 'model']:
    data[i] = data[i].map(
        dict(zip(data[i].unique(), range(data[i].nunique()))))
data['mt'] = (data['regYear'] - 2016) * 12 + data['regMonth']
data.head()

,adcode,bodyType,forecastVolum,id,model,province,regMonth,regYear,salesVolume,popularity,carCommentVolum,newsReplyVolum,label,is,mt
0,310000,0,NaN,NaN,0,上海,1,2016,292.0,1479.0,11.0,106.0,292.0,False,1
1,530000,0,NaN,NaN,0,云南,1,2016,466.0,1594.0,11.0,106.0,466.0,False,1
2,150000,0,NaN,NaN,0,内蒙古,1,2016,257.0,1479.0,11.0,106.0,257.0,False,1
3,110000,0,NaN,NaN,0,北京,1,2016,408.0,2370.0,11.0,106.0,408.0,False,1
4,510000,0,NaN,NaN,0,四川,1,2016,610.0,3562.0,11.0,106.0,610.0,False,1


In [10]:
def get_stat_feature(df_):
    df = df_.copy()
    stat_feat = []
    df['model_adcode'] = df['adcode'] + df['model']
    df['model_adcode_mt'] = df['model_adcode'] * 100 + df['mt']
    for col in tqdm(['label', 'popularity']):
        # shift
        for i in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]:
            stat_feat.append('shift_model_adcode_mt_{}_{}'.format(col, i))
            df['model_adcode_mt_{}_{}'.format(
                col, i)] = df['model_adcode_mt'] + i
            df_last = df[~df[col].isnull()].set_index(
                'model_adcode_mt_{}_{}'.format(col, i))
            df_last.to_csv('df_last.csv')
            df['shift_model_adcode_mt_{}_{}'.format(
                col, i)] = df['model_adcode_mt'].map(df_last[col])
            # 固定滑窗长度为5
            # if i >= 3 and i <= 10:
            #     stat_feat.append('model_adcode_mt_{}_{}_mean'.format(col, i))
            #     df['model_adcode_mt_{}_{}_mean'.format(
            #         col, i)] = df.loc[:, ['shift_model_adcode_mt_{}_{}'.format(
            #             col, j) for j in range(i - 2, i + 3)]].mean(axis=1, skipna=False)
    # 销量滑窗特征
    # 前几个月的销量
    for month in [2, 3, 4, 5, 6, 8, 12]:
        df['mean_in_{}_months'.format(month)] = df.loc[:, ['shift_model_adcode_mt_label_{}'.format(
            i) for i in range(1, month)]].mean(axis=1, skipna=False)

    for month in [2, 3, 4, 5, 6, 8, 12]:
        df['std_in_{}_months'.format(month)] = df.loc[:, ['shift_model_adcode_mt_label_{}'.format(
            i) for i in range(1, month)]].std(axis=1, skipna=False)

    #     # 添加进特征list
    for i in [2, 3, 4, 5, 6, 8, 12]:
        stat_feat.append('mean_in_{}_months'.format(i))
        stat_feat.append('std_in_{}_months'.format(i))
    
    # 添加是否春节字段
    df['is_spring_featival'] = df['regMonth'] == 2

    # 添加每一个季度同车型同省份的销量均值和方差
    for month in [4, 7, 10, 13]:
        df['season_sales_mean'.format(month - 1)] = df['regMonth'].between(month - 3, month).mean()
        df['season_sales_std'.format(month - 1)] = df['regMonth'].between(month - 3, month).std()

    for month in [4, 7, 10, 13]:
        stat_feat.append('season_sales_mean'.format(month - 1))
        stat_feat.append('season_sales_std'.format(month - 1))

    return df, stat_feat

In [11]:
df, feat = get_stat_feature(data)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:09<00:00, 30.66s/it]
